In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import cufflinks as cf
import plotly.express as px
import chart_studio.plotly as py
import plotly.graph_objects as go
%matplotlib inline

from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)
cf.go_offline()


def scale_axis(data: pd.DataFrame, scale:dict) -> pd.DataFrame:
    data = data.copy()
    new_list = []
    for key in data["name"].to_list():
        new_list.append(scale[key])
    data["scale"] = new_list
    data["x"] = data["x"] / data["scale"]
    data["y"] = data["y"] / data["scale"]
    data["z"] = data["z"] / data["scale"]
    return data

def frame_args(duration):
    return {
            "frame": {"duration": duration},
            "mode": "next",
            "fromcurrent": True,
            "transition": {"duration": duration, "easing": "linear"},
            }   


In [ ]:
AU = 149597870700.0

planets_data = pd.read_csv("planets_data.csv", index_col=0)
planets_data = planets_data[planets_data.name != "Venus"]
number_of_planets = planets_data.shape[0]
planets_data["radius"][planets_data["name"] == "Sun"] = \
    planets_data["radius"][planets_data["name"] == "Sun"]/40000
planets_data["radius"] =  np.sqrt(planets_data["radius"]) / 350
planets_data

In [ ]:
import datetime

# scale = {"Sun": 8e8, "Mercury": 8e8, "Venus":1e9, "Earth":1e9, "Mars":1e9,
#          "Jupiter":2.5e9, "Saturn":3e9, "Uranus":4e9, "Neptune": 4e9}
scale = {"Sun": 1, "Mercury": 0.5, "Venus":0.5, "Earth":0.5, "Mars":0.5,
         "Jupiter":1, "Saturn":1, "Uranus":1.3, "Neptune": 1.3}

position_history = pd.read_csv("results_wide.csv", index_col=0)
position_history["time"] = pd.to_datetime(position_history["time"]).astype("datetime64[D]")
position_history["time"] = position_history["time"].dt.strftime("%d:%m:%Y")

position_history.drop(labels=["Venus_x", "Venus_y", "Venus_z"], axis=1 ,inplace=True)

# Scaling data according to scale dictionary
for planet_name in planets_data["name"].to_list():
    position_history[planet_name + "_x"] = position_history[planet_name + "_x"] / AU / scale[planet_name] # 
    position_history[planet_name + "_y"] = position_history[planet_name + "_y"] / AU / scale[planet_name] # 
    position_history[planet_name + "_z"] = position_history[planet_name + "_z"] / AU / scale[planet_name] # 
    
x_limit = position_history["Neptune_x"].abs().max()
y_limit = position_history["Neptune_y"].abs().max()
z_limit = position_history["Neptune_z"].abs().max()

max_dim = max(x_limit, y_limit, z_limit)
max_dim = [-1.1*max_dim, 1.1*max_dim]

print(f"Simulation spans: {position_history.shape[0] : >5} [days]")
# position_history.head()

In [ ]:
# draw spherical object
sphere_resolution = 12 #the higher resolution the longer initial render time and slower animation

def sphere(size, position, name,color): 
    # Set up 100 points. First, do angles
    theta = np.linspace(0,2*np.pi,sphere_resolution)
    phi = np.linspace(0,np.pi,sphere_resolution)
    
    # Set up coordinates for points on the sphere
    x0 = position[0] + size * np.outer(np.cos(theta),np.sin(phi))
    y0 = position[1] + size * np.outer(np.sin(theta),np.sin(phi))
    z0 = position[2] + size * np.outer(np.ones(sphere_resolution),np.cos(phi))
#     print(x0)
    
    # Set up spherical trace
    trace= go.Surface(x=x0, y=y0, z=z0, colorscale=[[0,color], [1,color]],name=name,showlegend=True)
    trace.update(showscale=False,showlegend=True)

    return trace

In [ ]:
time_step = 10

initial_image = []
for planet_name in planets_data["name"].to_list():
    planets_data_row = np.dot(planets_data.index, planets_data["name"] == planet_name)
    initial_image.append(go.Scatter3d(x=[position_history[planet_name+"_x"][0]],
                                      y=[position_history[planet_name+"_y"][0]],
                                      z=[position_history[planet_name+"_z"][0]],
                                      mode="lines",
                                      showlegend=False,
                                      marker=dict(color="grey"),
                                      )
                        )
    initial_image.append(sphere(size=planets_data["radius"][planets_data_row],
                             position=[position_history[planet_name+"_x"][0],
                                      position_history[planet_name+"_y"][0],
                                      position_history[planet_name+"_z"][0]],
                             name=planet_name,
                             color=planets_data["color"][planets_data_row])
                         )

frames = []
for frame_number in range(1, position_history.shape[0], time_step):
    traces = []
    for planet_name in planets_data["name"].to_list():
        planets_data_row = np.dot(planets_data.index, planets_data["name"] == planet_name)

        orbit_start_frame = max([int(frame_number - planets_data["orbit_time"][planets_data_row]), 0])
        traces.append(go.Scatter3d(x=position_history[planet_name+"_x"][orbit_start_frame:frame_number],
                                   y=position_history[planet_name+"_y"][orbit_start_frame:frame_number],
                                   z=position_history[planet_name+"_z"][orbit_start_frame:frame_number],
                                   mode="lines",
                                   showlegend=False,
                                   marker=dict(color="grey"),
                                   )
                     )

#         traces.append(go.Scatter3d(x=[position_history[planet_name+"_x"][frame_number]],
#                                    y=[position_history[planet_name+"_y"][frame_number]],
#                                    z=[position_history[planet_name+"_z"][frame_number]],
#                                    name=planet_name,
#                                    mode="markers",
#                                    marker=dict(color=planets_data["color"][planets_data_row],
#                                                size=planets_data["radius"][planets_data_row])
#                                    )
#                      )
        traces.append(sphere(size=planets_data["radius"][planets_data_row],
                             position=[position_history[planet_name+"_x"][frame_number],
                                       position_history[planet_name+"_y"][frame_number],
                                       position_history[planet_name+"_z"][frame_number]],
                             name=planet_name,
                             color=planets_data["color"][planets_data_row]))
    
    frames.append(go.Frame(data=traces,name=frame_number))

In [ ]:
fig = go.Figure(frames=frames)
fig.add_traces(initial_image)

update_menus = [
                {
                    "buttons": [
                        {
                            "args": [None, frame_args(30)],
                            "label": "&#9654;", # play symbol
                            "method": "animate",
                        },
                        {
                            "args": [[None], frame_args(0)],
                            "label": "&#9724;", # pause symbol
                            "method": "animate",
                        },
                    ],
                        "direction": "left",
                        "pad": {"r": 10, "t": 70},
                        "type": "buttons",
                        "x": 0.1,
                        "y": 0,
                }
            ]

sliders = [
            {
                "pad": {"b": 10, "t": 60},
                "len": 0.9,
                "x": 0.1,
                "y": 0,
                "name": "Animaton step",
                "steps": [
                    {
                        "args": [[frame.name], frame_args(0)],
                        "label": "Day "+str(step*time_step),
                        "method": "animate",
                    }
                    for step, frame in enumerate(fig.frames)
                ],
            }
        ]



In [ ]:
fig.update_layout(title="Solar system simulation [SpaceX approved]",
                  scene=dict(xaxis = dict(title="x [AU]",range=max_dim, autorange=False, 
                                          visible=True, showbackground=False, gridcolor="gray"),
                             yaxis = dict(title="y [AU]",range=max_dim, autorange=False, 
                                          visible=True, showbackground=False, gridcolor="gray"),
                             zaxis = dict(title="z [AU]",range=max_dim, autorange=False, 
                                          visible=True, showbackground=False, gridcolor="gray"),
                             aspectratio=dict(x=1, y=1, z=1),
                             camera=dict(eye=dict(x=-0.6, y=-0.1, z=0.6)),
                             bgcolor="black",
                             ),
                  height=800,
                  updatemenus = update_menus,
                  sliders=sliders,
                  transition={"duration":33}
                  )
fig.show()

In [ ]:
fig.write_html("simulation.html")

## Before saving clear simulation (eg. turn cell to markdown) or the file will take 100mb

In [ ]:
scale = 1
name = "Sun"
print(f"{name : >8} x: {np.amax(position_history[name + '_x']) / scale : 8.2f}")
print(f"{name : >8} y: {np.amax(position_history[name + '_y']) / scale : 8.2f}")
print(f"{name : >8} z: {np.amax(position_history[name + '_z']) / scale : 8.2f}")
print("---------------------------------------------")

scale = 0.5
name = "Mercury"
print(f"{name : >8} x: {np.amin(position_history[name + '_x']) / scale: 8.2f}")
print(f"{name : >8} y: {np.amin(position_history[name + '_y']) / scale: 8.2f}")
print(f"{name : >8} z: {np.amin(position_history[name + '_z']) / scale: 8.2f}")
print("---------------------------------------------")

scale = 0.5
name = "Earth"
print(f"{name : >8} x: {np.amin(position_history[name + '_x']) / scale: 8.2f}")
print(f"{name : >8} y: {np.amin(position_history[name + '_y']) / scale: 8.2f}")
print(f"{name : >8} z: {np.amin(position_history[name + '_z']) / scale: 8.2f}")
print("---------------------------------------------")

scale = 0.5
name = "Mars"
print(f"{name : >8} x: {np.amin(position_history[name + '_x']) / scale: 8.2f}")
print(f"{name : >8} y: {np.amin(position_history[name + '_y']) / scale: 8.2f}")
print(f"{name : >8} z: {np.amin(position_history[name + '_z']) / scale: 8.2f}")
print("---------------------------------------------")

scale = 1
name = "Jupiter"
print(f"{name : >8} x: {np.amin(position_history[name + '_x']) / scale: 8.2f}")
print(f"{name : >8} y: {np.amin(position_history[name + '_y']) / scale: 8.2f}")
print(f"{name : >8} z: {np.amin(position_history[name + '_z']) / scale: 8.2f}")
print("---------------------------------------------")

scale = 1
name = "Saturn"
print(f"{name : >8} x: {np.amin(position_history[name + '_x']) / scale: 8.2f}")
print(f"{name : >8} y: {np.amin(position_history[name + '_y']) / scale: 8.2f}")
print(f"{name : >8} z: {np.amin(position_history[name + '_z']) / scale: 8.2f}")
print("---------------------------------------------")

scale = 1.5
name = "Uranus"
print(f"{name : >8} x: {np.amin(position_history[name + '_x']) / scale: 8.2f}")
print(f"{name : >8} y: {np.amin(position_history[name + '_y']) / scale: 8.2f}")
print(f"{name : >8} z: {np.amin(position_history[name + '_z']) / scale: 8.2f}")
print("---------------------------------------------")

scale = 1.5
name = "Neptune"
print(f"{name : >8} x: {np.amin(position_history[name + '_x']) / scale: 8.2f}")
print(f"{name : >8} y: {np.amin(position_history[name + '_y']) / scale: 8.2f}")
print(f"{name : >8} z: {np.amin(position_history[name + '_z']) / scale: 8.2f}")
print("---------------------------------------------")